In [48]:
import os
import csv
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from scipy.ndimage import uniform_filter1d
from scipy.signal import butter, filtfilt

import numpy as np

import re
import pickle

from dataset import MotionDataset
from time_model import TimeModel
from tqdm.notebook import tqdm

data_path = "/home/meribejayson/Desktop/Projects/realistic-imu/data/total_capture_data"
subjects = ["s5"]

minimized_dataset = MotionDataset(data_path, subjects=subjects, dataset_type="minimized", minimize=True)
minimized_loader = DataLoader(minimized_dataset, batch_size=1, shuffle=False)

orig_dataset = MotionDataset(data_path, subjects=subjects, dataset_type="orig", minimize=False)
orig_loader = DataLoader(orig_dataset, batch_size=1, shuffle=False)

## Mean Squared Error of only applying a finite difference

In [49]:
num_exps = 0
total = 0

for mocap, imu in tqdm(orig_loader):
    diff = imu[0, :, :] - mocap[0, :, 0:26:2]
    sqr = diff ** 2

    num_exps += sqr.shape[0] * sqr.shape[1]

    total += torch.sum(sqr)

total /= num_exps
total.item()

  0%|          | 0/5 [00:00<?, ?it/s]

12.683247566223145

## Mean Squared Error with minimized accelerations

In [50]:
num_exps = 0
total = 0

for mocap, imu in tqdm(minimized_loader):
    diff = imu[0, :, :] - mocap[0, :, 0:26:2]
    sqr = diff ** 2

    num_exps += sqr.shape[0] * sqr.shape[1]

    total += torch.sum(sqr)

total /= num_exps
total.item()

  0%|          | 0/5 [00:00<?, ?it/s]

11.728960990905762

## Mean Squared error with moving average

In [51]:
num_exps = 0
total = 0

for mocap, imu in tqdm(orig_loader):
    avg_mocap = uniform_filter1d(mocap[0, :, 0:26:2].numpy(), size=11, mode='nearest', axis=0)
    diff = imu[0, :, :] - avg_mocap
    sqr = diff ** 2
    num_exps += sqr.shape[0] * sqr.shape[1]

    total += torch.sum(sqr)

total /= num_exps
total.item()

  0%|          | 0/5 [00:00<?, ?it/s]

13.127171516418457

## Mean Squared error with butterworth filter

In [52]:
def butterworth_filter(data, cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data, axis=0)

cutoff = 10
fs = 60
order = 1

num_exps = 0
total = 0

for mocap, imu in tqdm(orig_loader):
    avg_mocap = butterworth_filter(mocap[0, :, 0:26:2].numpy(), cutoff, fs, order)
    diff = imu[0, :, :] - avg_mocap
    sqr = diff ** 2
    num_exps += sqr.shape[0] * sqr.shape[1]

    total += torch.sum(sqr)

total /= num_exps
total.item()

  0%|          | 0/5 [00:00<?, ?it/s]

11.479291288241145

## Mean Squared Error of finite differenced smpl vertices